# Usage suggestions for this python package

First install the package:

Activate the python environment you want to use. Go into the folder of this notebook, where setup.py located is, and run 

`pip install -e .`

In [1]:
import relational_image_generation_evaluation as rige

In [2]:
evaluator = rige.Evaluator('ViT-B/32')
# testonly just makes loading faster, to actually test the model you need to set testonly=False
# this is not the test set or so. It's just a flag to make loading faster
dataloader_one = rige.get_one_edge_dataloader(testonly=True)
dataloader_two = rige.get_two_edge_dataloader(testonly=True)
print("len(dataloader_one):", len(dataloader_one))
print("len(dataloader_two):", len(dataloader_two))

Using device cuda:1 for evaluation.
Loading filtered test graphs...
Finished loading filtered test graphs
Generating one edge graphs...


100%|██████████| 100/100 [00:00<00:00, 1132.28it/s]


Finished generating one edge graphs
Loading filtered test graphs...
Finished loading filtered test graphs
Generating two edge graphs...


100%|██████████| 100/100 [00:00<00:00, 900.69it/s]

Finished generating two edge graphs
len(dataloader_one): 837
len(dataloader_two): 1076


**The dataloaders give you networkx graphs as they have convenient functionality.**

**You can access all scene information like this (images have to be loaded using PATH_TO_VG/image_id.jpg):**

In [3]:
some_graph = next(iter(dataloader_two))[0]
print("nodes:", some_graph.nodes)
print("edges:", some_graph.edges)
edge = list(some_graph.edges)[0]
print("some edge:", edge, "edge predicate:", some_graph.edges[edge]['predicate'])
node = list(some_graph.nodes)[0]
print("some node:", node, "node label:", some_graph.nodes[node]['name'])
# other information:
print(some_graph.nodes[node].keys())
print(some_graph.edges[edge].keys())
print("image_id:", some_graph.image_id)
print("image dimensions:", some_graph.image_w, some_graph.image_h)
# use this copy function to make a copy of the graph, because much faster than deepcopy and normal copy doesn't copy everything
some_graph_copy = rige.copy_graph(some_graph)

nodes: [1025157, 1025159, 5890]
edges: [(1025159, 1025157), (5890, 1025157)]
some edge: (1025159, 1025157) edge predicate: on
some node: 1025157 node label: wall
dict_keys(['w', 'h', 'x', 'y', 'attributes', 'name'])
dict_keys(['synsets', 'relationship_id', 'predicate'])
image_id: 39
image dimensions: 800 534


In [4]:
from PIL import Image
images = []
graphs = []
for i in range(10):
    graph = next(iter(dataloader_one))[0]
    image_id = graph.image_id
    # adapt to your local directory
    IMAGE_DIR = '/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/VG/'
    image = Image.open(IMAGE_DIR + str(image_id) + '.jpg')
    images.append(image)
    graphs.append(graph)

The ViT finetuned models: For each graph you evaluate they give you a relationship probability confidence and an attribute probability confidence. Both are the mean confidences for the relationships and attributes of the graph you give in.

In many graphs, there are no attributes, in this case, 'noattributes' os given back instead of a numerical score. Same behaviour for edges.

In [5]:
scores = evaluator(images,graphs)
print(scores)
print(scores.keys())

{'rel_scores': [0.3381844162940979, 0.016545556485652924, 0.7016329169273376, 0.38774237036705017, 0.03416311368346214, 0.012142871506512165, 0.7594617605209351, 0.6007148623466492, 0.01968073472380638, 0.6791365146636963], 'attr_scores': [0.756584882736206, 0.7096579670906067, 'noattributes', 'noattributes', 0.4308685064315796, 0.3934137225151062, 'noattributes', 0.8246681094169617, 0.5452541708946228, 0.956260621547699]}
dict_keys(['rel_scores', 'attr_scores'])
